In [9]:
# Import Necessary Libraries

In [1]:
import json 
import pandas as pd
pd.set_option('display.max_rows', 500)
# pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
import requests
import ast
response = requests.get("http://api.open-notify.org/astros.json")
print(response)

<Response [200]>


In [3]:
# The following parameters are used to request data for ISS on specified latitude and longitude
query = {'lat':'45', 'lon':'180'}
response = requests.get('http://api.open-notify.org/iss-pass.json', params=query)
print(response.json())

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1648593101, 'latitude': 45.0, 'longitude': 180.0, 'passes': 5}, 'response': [{'duration': 548, 'risetime': 1648604717}, {'duration': 654, 'risetime': 1648610443}, {'duration': 633, 'risetime': 1648616276}, {'duration': 630, 'risetime': 1648622122}, {'duration': 656, 'risetime': 1648627937}]}


In [ ]:
# Convert HTTP response to Dataframe

In [5]:
data = response.json()

In [6]:
data

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1648593101,
  'latitude': 45.0,
  'longitude': 180.0,
  'passes': 5},
 'response': [{'duration': 548, 'risetime': 1648604717},
  {'duration': 654, 'risetime': 1648610443},
  {'duration': 633, 'risetime': 1648616276},
  {'duration': 630, 'risetime': 1648622122},
  {'duration': 656, 'risetime': 1648627937}]}

In [7]:
data.keys()

dict_keys(['message', 'request', 'response'])

In [10]:
data1 = json.dumps(data)

In [11]:
json.loads(data1)

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1648593101,
  'latitude': 45.0,
  'longitude': 180.0,
  'passes': 5},
 'response': [{'duration': 548, 'risetime': 1648604717},
  {'duration': 654, 'risetime': 1648610443},
  {'duration': 633, 'risetime': 1648616276},
  {'duration': 630, 'risetime': 1648622122},
  {'duration': 656, 'risetime': 1648627937}]}

In [20]:
df = pd.json_normalize(data)

In [21]:
df.head()

,message,response,request.altitude,request.datetime,request.latitude,request.longitude,request.passes
0,success,"[{'duration': 548, 'risetime': 1648604717}, {'...",100,1648593101,45.0,180.0,5


In [ ]:
# Creating secondary dataframe to expand response column

In [22]:
dff = pd.concat([pd.DataFrame(x) for x in df['response']], keys=df['request.datetime']).reset_index(level=1, drop=True).reset_index()

In [23]:
dff

,request.datetime,duration,risetime
0,1648593101,548,1648604717
1,1648593101,654,1648610443
2,1648593101,633,1648616276
3,1648593101,630,1648622122
4,1648593101,656,1648627937


In [ ]:
# Merging two dataframes

In [26]:
df = df.merge(dff, on='request.datetime', how='left')

In [27]:
df.head()

,message,response,request.altitude,request.datetime,request.latitude,request.longitude,request.passes,duration,risetime
0,success,"[{'duration': 548, 'risetime': 1648604717}, {'...",100,1648593101,45.0,180.0,5,548,1648604717
1,success,"[{'duration': 548, 'risetime': 1648604717}, {'...",100,1648593101,45.0,180.0,5,654,1648610443
2,success,"[{'duration': 548, 'risetime': 1648604717}, {'...",100,1648593101,45.0,180.0,5,633,1648616276
3,success,"[{'duration': 548, 'risetime': 1648604717}, {'...",100,1648593101,45.0,180.0,5,630,1648622122
4,success,"[{'duration': 548, 'risetime': 1648604717}, {'...",100,1648593101,45.0,180.0,5,656,1648627937


In [28]:
df = df.drop(['response'], axis = 1)

In [29]:
df

,message,request.altitude,request.datetime,request.latitude,request.longitude,request.passes,duration,risetime
0,success,100,1648593101,45.0,180.0,5,548,1648604717
1,success,100,1648593101,45.0,180.0,5,654,1648610443
2,success,100,1648593101,45.0,180.0,5,633,1648616276
3,success,100,1648593101,45.0,180.0,5,630,1648622122
4,success,100,1648593101,45.0,180.0,5,656,1648627937


In [ ]:
# Export cleaned data

In [37]:
df.to_csv('output.csv', index=False)

In [ ]:
# Reading exported data from the disk

In [38]:
output_file = pd.read_csv('output.csv')

In [39]:
output_file.head()

,message,request.altitude,request.datetime,request.latitude,request.longitude,request.passes,duration,risetime
0,success,100,1648593101,45.0,180.0,5,548,1648604717
1,success,100,1648593101,45.0,180.0,5,654,1648610443
2,success,100,1648593101,45.0,180.0,5,633,1648616276
3,success,100,1648593101,45.0,180.0,5,630,1648622122
4,success,100,1648593101,45.0,180.0,5,656,1648627937
